In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=UserWarning, module='h2o')
import pandas.api.types as ptypes
import pickle
from copy import deepcopy
import h2o
import os

In [2]:
def Project_2_Scoring(testdata):
    holdoutset = testdata.copy()
    artifacts_dict_file = open("../artifacts/artifacts_dict_file.pkl", "rb")
    artifacts_dict = pickle.load(file=artifacts_dict_file)
    artifacts_dict_file.close()
    artifacts_functions_file = open("../artifacts/artifacts_functions_file.pkl", "rb")
    artifacts_functions = pickle.load(file=artifacts_functions_file)
    artifacts_functions_file.close()
    exec(artifacts_functions["get_naics"], globals())
    exec(artifacts_functions["fillMissingValues"], globals())
    exec(artifacts_functions["data_clean_preprocess"], globals())
    exec(artifacts_functions["create_engineered_features"], globals())
    data_clean_preprocess(holdoutset)
    create_engineered_features(holdoutset,artifacts_dict)
    h2o.init(max_mem_size = "4G",nthreads=4)            
    h2o.remove_all() 
    h2o_hold = h2o.H2OFrame(holdoutset)
    h2o_hold[artifacts_dict["CategoricalColumns"]] = h2o_hold[artifacts_dict["CategoricalColumns"]].asfactor()
    saved_model = h2o.load_model('../artifacts/'+artifacts_dict["ModelId"])
    predictions = saved_model.predict(h2o_hold)
    #converting h2oframe to pandas dataframe by this method because as_data_frame throws an error
    h2o.export_file(predictions, path="temp.csv", force=True)
    pred_df = pd.read_csv("temp.csv")
    output_df = pd.DataFrame()
    output_df['index']= holdoutset[['index']]
    output_df['label'] = (pred_df['p1'] >= artifacts_dict["threshold"]).astype(int) # Converting predictions to integer labels
    output_df['probability_0'] = pred_df['p0']
    output_df['probability_1'] = pred_df['p1']
    os.remove("temp.csv")
    return output_df

In [3]:
testdata = pd.read_csv("data/SBA_loans_project_2_holdout_students_valid.csv")
scoreddata = Project_2_Scoring(testdata)
scoreddata.head()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 17+35-LTS-2724, mixed mode, sharing)
  Starting server from C:\Users\megha\OneDrive\Desktop\AdvancedML\PythonEnv\ml-summer-2024\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\megha\AppData\Local\Temp\tmpucmvd4j6
  JVM stdout: C:\Users\megha\AppData\Local\Temp\tmpucmvd4j6\h2o_megha_started_from_python.out
  JVM stderr: C:\Users\megha\AppData\Local\Temp\tmpucmvd4j6\h2o_megha_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,7 months and 18 days
H2O_cluster_name:,H2O_from_python_megha_k8e1gu
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Export File progress: |██████████████████████████████████████████████████████████| (done) 100%


,index,label,probability_0,probability_1
0,0,0,0.950855,0.049145
1,1,0,0.902757,0.097243
2,2,0,0.926519,0.073481
3,3,0,0.977844,0.022156
4,4,0,0.945210,0.054790


#### Since as_data_frame doesnt work on windows. I have exported the h2o dataframe into a csv file and reading it again into a pandas dataframe to return the output. So it is important to make sure to shutdown all kernels and H2O instances before running this code incase there is an error that temp.csv isnt found.